# OVERLAPPING VARIANTS
### 7. Variants that are overlapping

This notebook contains the code that generated Table 1 in the original manuscript.

...



In [1]:
%load_ext autoreload
%autoreload 2

from utils import *
import os

In [2]:
DIRECTORY = os.getcwd()

In [3]:
all_vcfs ={}

pipelines = ["mutect_yb_bwa", 
             "mutect_yb_bowtie", 
             "mutect_nb_bwa", 
             "mutect_nb_bowtie", 
             "strelka_yb_bwa", 
             "strelka_yb_bowtie", 
             "strelka_nb_bwa", 
             "strelka_nb_bowtie",
             "ss_nb_bowtie",
             "ss_nb_bwa",
             "ss_yb_bowtie",
             "ss_yb_bwa"]

# "yb" and "nb" represent "presence" and "absence" of the base recalibration step
# "bwa" and "bowtie" represent the aligner used
# "mutect", "strelka", and "ss (somaticsniper)" represent the variant caller used.

hc_vcf = os.path.join(DIRECTORY, "hc_bed_filtered.recode.vcf")
# "hc" represents the high-confidence variant list used

In [4]:
for pipline in iter(pipelines):
    all_vcfs[pipline] = read_all_vcfs(os.path.join(DIRECTORY, pipline))

In [5]:
def get_group(filename):
    group = filename.split("_")[0]
    return group[0].upper() + group[1:]


def get_pipeline(pipeline_name):
    pipeline_name = pipeline_name.replace("_", "-")
    pipeline_name = pipeline_name.replace("bwa", "BWA")
    pipeline_name = pipeline_name.replace("bowtie", "Bowtie")
    pipeline_name = pipeline_name.replace("ss", "SS")
    pipeline_name = pipeline_name.replace("mutect", "Mutect")
    pipeline_name = pipeline_name.replace("strelka", "Strelka")
    pipeline_name = pipeline_name.replace("yb", "YB")
    pipeline_name = pipeline_name.replace("nb", "NB")

    return pipeline_name


def format_cell_value(variant_count, jaccard_percentage):
    """Format cell value with variant count and Jaccard similarity percentage"""
    return f"{variant_count} ({jaccard_percentage:.1f}%)"


In [6]:
# Parse the high-confidence VCF file once (ground truth)
print("Parsing high-confidence VCF file...")
true_vcf = parse_vcf(os.path.join(DIRECTORY, hc_vcf))
print(f"High-confidence VCF contains {len(true_vcf)} variants")
print("High-confidence VCF parsing completed.")

Parsing high-confidence VCF file...
High-confidence VCF contains 1161 variants
High-confidence VCF parsing completed.


In [7]:
# Create a table that contains both variant counts and Jaccard similarity values
print("Processing VCF files and calculating Jaccard similarities...")

variant_counts = {'Pipeline': [], 'Group': [], 'Variant Count': [], 'Jaccard Similarity': []}

for pipeline in pipelines:
    print(f"Processing pipeline: {pipeline}")
    for i in range(len(all_vcfs[pipeline])):
        # Parse the current VCF file
        vcf_file_path = os.path.join(DIRECTORY, pipeline, all_vcfs[pipeline][i])
        vcf_file = parse_vcf(vcf_file_path)
        
        # Calculate variant count
        variant_count = len(vcf_file)
        
        # Calculate Jaccard similarity with high-confidence variants
        jaccard_sim = jaccard_similarity(vcf_file, true_vcf)
        jaccard_percentage = jaccard_sim * 100
        
        # Store the data
        variant_counts['Pipeline'].append(get_pipeline(pipeline))
        variant_counts['Group'].append(get_group(all_vcfs[pipeline][i]))
        variant_counts['Variant Count'].append(variant_count)
        variant_counts['Jaccard Similarity'].append(jaccard_percentage)
        
        print(f"  {all_vcfs[pipeline][i]}: {variant_count} variants, {jaccard_percentage:.1f}% Jaccard similarity")

print("\nAll VCF files processed successfully!")

Processing VCF files and calculating Jaccard similarities...
Processing pipeline: mutect_yb_bwa
  g10_mutect_yb_bwa.vcf: 489 variants, 22.4% Jaccard similarity
  g1_mutect_yb_bwa.vcf: 1052 variants, 69.4% Jaccard similarity
  g4_mutect_yb_bwa.vcf: 2735 variants, 24.6% Jaccard similarity
  g11_mutect_yb_bwa.vcf: 144 variants, 8.4% Jaccard similarity
  g5_mutect_yb_bwa.vcf: 838 variants, 55.2% Jaccard similarity
  g3_mutect_yb_bwa.vcf: 851 variants, 36.8% Jaccard similarity
  g6_mutect_yb_bwa.vcf: 663 variants, 42.8% Jaccard similarity
  g2_mutect_yb_bwa.vcf: 1044 variants, 69.4% Jaccard similarity
  g7_mutect_yb_bwa.vcf: 1050 variants, 69.4% Jaccard similarity
  g9_mutect_yb_bwa.vcf: 322 variants, 21.4% Jaccard similarity
  g8_mutect_yb_bwa.vcf: 663 variants, 42.8% Jaccard similarity
Processing pipeline: mutect_yb_bowtie
  g6_mutect_yb_bowtie.vcf: 477 variants, 34.0% Jaccard similarity
  g3_mutect_yb_bowtie.vcf: 734 variants, 37.3% Jaccard similarity
  g9_mutect_yb_bowtie.vcf: 324 varia

In [8]:
# Convert to DataFrame and create pivot tables
variant_counts_df = pd.DataFrame(variant_counts)

# Create pivot table for variant counts
pivot_variant_counts = variant_counts_df.pivot_table(
    index='Pipeline', 
    columns='Group', 
    values='Variant Count', 
    aggfunc='first'
)

# Create pivot table for Jaccard similarities
pivot_jaccard = variant_counts_df.pivot_table(
    index='Pipeline', 
    columns='Group', 
    values='Jaccard Similarity', 
    aggfunc='first'
)

print("Pivot tables created successfully!")

Pivot tables created successfully!


In [9]:
# Display the variant counts table
print("=== VARIANT COUNTS TABLE ===")
display(pivot_variant_counts)

print("\n=== JACCARD SIMILARITY TABLE (Percentages) ===")
display(pivot_jaccard)

=== VARIANT COUNTS TABLE ===


Group,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Pipeline,,,,,,,,,,,
Mutect-NB-BWA,926,435,235,715,518,4103,771,745,1078,745,275
Mutect-NB-Bowtie,880,1819,381,892,668,3566,385,485,952,485,233
Mutect-YB-BWA,1052,489,144,1044,851,2735,838,663,1050,663,322
Mutect-YB-Bowtie,637,1830,246,588,734,2836,939,477,1004,477,324
SS-NB-BWA,2889,8480,8480,2889,9397,2889,2889,2889,2889,2889,2889
SS-NB-Bowtie,2406,6137,6858,2406,6858,2406,2406,2406,2406,2406,2406
SS-YB-BWA,2313,7492,7492,2313,8290,1890,2313,2313,2313,2312,2312
SS-YB-Bowtie,1752,4934,5521,1752,5521,1752,1752,1752,1752,1752,1752
Strelka-NB-BWA,2204,2832,2898,2204,3169,108864,2204,2204,2203,2201,2201



=== JACCARD SIMILARITY TABLE (Percentages) ===


Group,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Pipeline,,,,,,,,,,,
Mutect-NB-BWA,52.670080,18.750000,12.853678,44.418784,22.376093,21.542369,50.350195,45.274390,67.841079,45.274390,19.467554
Mutect-NB-Bowtie,56.039755,9.357798,20.941176,63.977636,32.921512,23.808277,28.940784,33.387358,68.500797,33.387358,17.637131
Mutect-YB-BWA,69.448698,22.403561,8.388704,69.354839,36.777702,24.592261,55.201863,42.834769,69.425287,42.834769,21.358429
Mutect-YB-Bowtie,45.117030,15.571870,14.297319,44.306931,37.318841,29.646448,69.628433,34.042553,67.959659,34.042553,23.646961
SS-NB-BWA,27.358491,8.508723,8.508723,27.358491,8.980182,27.358491,27.358491,27.358491,27.358491,27.358491,27.358491
SS-NB-Bowtie,29.098806,10.525519,11.143451,29.098806,11.143451,29.098806,29.098806,29.098806,29.098806,29.098806,29.098806
SS-YB-BWA,32.595420,9.351700,9.351700,32.595420,9.933698,30.496151,32.595420,32.595420,32.595420,32.607866,32.607866
SS-YB-Bowtie,36.632270,12.412394,13.235045,36.632270,13.235045,36.632270,36.632270,36.632270,36.632270,36.632270,36.632270
Strelka-NB-BWA,38.934765,25.843051,25.316456,38.934765,27.841748,0.945924,38.934765,38.934765,38.950847,38.983051,38.983051


In [10]:
# Create the combined table with both variant counts and Jaccard similarities
print("Creating combined table with variant counts and Jaccard similarities...")

# Initialize the combined table
combined_data = {}
pipeline_names = pivot_variant_counts.index.tolist()
group_names = pivot_variant_counts.columns.tolist()

# Create combined table structure
for pipeline in pipeline_names:
    combined_data[pipeline] = {}
    for group in group_names:
        variant_count = pivot_variant_counts.loc[pipeline, group]
        jaccard_pct = pivot_jaccard.loc[pipeline, group]
        
        if pd.isna(variant_count) or pd.isna(jaccard_pct):
            combined_data[pipeline][group] = "N/A"
        else:
            combined_data[pipeline][group] = format_cell_value(variant_count, jaccard_pct)

# Convert to DataFrame
combined_df = pd.DataFrame(combined_data).T

print("Combined table created successfully!")
display(combined_df)

Creating combined table with variant counts and Jaccard similarities...
Combined table created successfully!


,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Mutect-NB-BWA,926 (52.7%),435 (18.8%),235 (12.9%),715 (44.4%),518 (22.4%),4103 (21.5%),771 (50.4%),745 (45.3%),1078 (67.8%),745 (45.3%),275 (19.5%)
Mutect-NB-Bowtie,880 (56.0%),1819 (9.4%),381 (20.9%),892 (64.0%),668 (32.9%),3566 (23.8%),385 (28.9%),485 (33.4%),952 (68.5%),485 (33.4%),233 (17.6%)
Mutect-YB-BWA,1052 (69.4%),489 (22.4%),144 (8.4%),1044 (69.4%),851 (36.8%),2735 (24.6%),838 (55.2%),663 (42.8%),1050 (69.4%),663 (42.8%),322 (21.4%)
Mutect-YB-Bowtie,637 (45.1%),1830 (15.6%),246 (14.3%),588 (44.3%),734 (37.3%),2836 (29.6%),939 (69.6%),477 (34.0%),1004 (68.0%),477 (34.0%),324 (23.6%)
SS-NB-BWA,2889 (27.4%),8480 (8.5%),8480 (8.5%),2889 (27.4%),9397 (9.0%),2889 (27.4%),2889 (27.4%),2889 (27.4%),2889 (27.4%),2889 (27.4%),2889 (27.4%)
SS-NB-Bowtie,2406 (29.1%),6137 (10.5%),6858 (11.1%),2406 (29.1%),6858 (11.1%),2406 (29.1%),2406 (29.1%),2406 (29.1%),2406 (29.1%),2406 (29.1%),2406 (29.1%)
SS-YB-BWA,2313 (32.6%),7492 (9.4%),7492 (9.4%),2313 (32.6%),8290 (9.9%),1890 (30.5%),2313 (32.6%),2313 (32.6%),2313 (32.6%),2312 (32.6%),2312 (32.6%)
SS-YB-Bowtie,1752 (36.6%),4934 (12.4%),5521 (13.2%),1752 (36.6%),5521 (13.2%),1752 (36.6%),1752 (36.6%),1752 (36.6%),1752 (36.6%),1752 (36.6%),1752 (36.6%)
Strelka-NB-BWA,2204 (38.9%),2832 (25.8%),2898 (25.3%),2204 (38.9%),3169 (27.8%),108864 (0.9%),2204 (38.9%),2204 (38.9%),2203 (39.0%),2201 (39.0%),2201 (39.0%)
Strelka-NB-Bowtie,1313 (50.6%),107672 (0.8%),1839 (38.3%),1313 (50.6%),1786 (39.3%),90416 (1.1%),1313 (50.6%),1313 (50.6%),1313 (50.6%),1313 (50.6%),1313 (50.6%)


In [11]:
# Save all tables to Excel file with multiple sheets
print("Saving results to Excel file...")

with pd.ExcelWriter("variant_counts_and_jaccard_similarity.xlsx", engine='openpyxl') as writer:
    # Sheet 1: Combined table (main results)
    combined_df.to_excel(writer, sheet_name='Combined_Results')
    
    # Sheet 2: Variant counts only
    pivot_variant_counts.to_excel(writer, sheet_name='Variant_Counts_Only')
    
    # Sheet 3: Jaccard similarities only
    pivot_jaccard.to_excel(writer, sheet_name='Jaccard_Similarities_Only')
    
    # Sheet 4: Raw data
    variant_counts_df.to_excel(writer, sheet_name='Raw_Data', index=False)

print("Results saved to 'variant_counts_and_jaccard_similarity.xlsx' with multiple sheets!")

Saving results to Excel file...
Results saved to 'variant_counts_and_jaccard_similarity.xlsx' with multiple sheets!


In [12]:
# Summary statistics
print("=== SUMMARY STATISTICS ===")
print(f"Total pipelines analyzed: {len(pipelines)}")
print(f"Total student groups: {len(group_names)}")
print(f"Total VCF files processed: {len(variant_counts_df)}")
print(f"High-confidence variants: {len(true_vcf)}")

# Calculate overall statistics
jaccard_values = variant_counts_df['Jaccard Similarity'].dropna()
print(f"\nJaccard Similarity Statistics:")
print(f"  Mean: {jaccard_values.mean():.2f}%")
print(f"  Median: {jaccard_values.median():.2f}%")
print(f"  Min: {jaccard_values.min():.2f}%")
print(f"  Max: {jaccard_values.max():.2f}%")
print(f"  Std Dev: {jaccard_values.std():.2f}%")

# Find best and worst performing pipelines
best_pipeline_idx = jaccard_values.idxmax()
worst_pipeline_idx = jaccard_values.idxmin()

print(f"\nBest performing pipeline: {variant_counts_df.loc[best_pipeline_idx, 'Pipeline']} - {variant_counts_df.loc[best_pipeline_idx, 'Group']} ({jaccard_values.max():.1f}%)")
print(f"Worst performing pipeline: {variant_counts_df.loc[worst_pipeline_idx, 'Pipeline']} - {variant_counts_df.loc[worst_pipeline_idx, 'Group']} ({jaccard_values.min():.1f}%)")

=== SUMMARY STATISTICS ===
Total pipelines analyzed: 12
Total student groups: 11
Total VCF files processed: 132
High-confidence variants: 1161

Jaccard Similarity Statistics:
  Mean: 33.71%
  Median: 32.76%
  Min: 0.81%
  Max: 69.63%
  Std Dev: 16.57%

Best performing pipeline: Mutect-YB-Bowtie - G5 (69.6%)
Worst performing pipeline: Strelka-NB-Bowtie - G10 (0.8%)


-----------

## PRECISION, RECALL, F1 SCORE

In [13]:
# Calculate precision, recall, and F1 scores for each VCF file
print("Calculating precision, recall, and F1 scores...")

# Initialize dictionaries for the new metrics
precision_data = {'Pipeline': [], 'Group': [], 'Precision': []}
recall_data = {'Pipeline': [], 'Group': [], 'Recall': []}
f1_data = {'Pipeline': [], 'Group': [], 'F1_Score': []}

for pipeline in pipelines:
    print(f"Processing pipeline: {pipeline}")
    for i in range(len(all_vcfs[pipeline])):
        # Parse the current VCF file
        vcf_file_path = os.path.join(DIRECTORY, pipeline, all_vcfs[pipeline][i])
        vcf_file = parse_vcf(vcf_file_path)
        
        # Calculate precision, recall, and F1 score
        precision, recall, f1, accuracy, tp, fp, fn, tn = calculate_metrics(vcf_file, true_vcf)
        
        # Store the data
        pipeline_name = get_pipeline(pipeline)
        group_name = get_group(all_vcfs[pipeline][i])
        
        precision_data['Pipeline'].append(pipeline_name)
        precision_data['Group'].append(group_name)
        precision_data['Precision'].append(precision * 100)  # Convert to percentage
        
        recall_data['Pipeline'].append(pipeline_name)
        recall_data['Group'].append(group_name)
        recall_data['Recall'].append(recall * 100)  # Convert to percentage
        
        f1_data['Pipeline'].append(pipeline_name)
        f1_data['Group'].append(group_name)
        f1_data['F1_Score'].append(f1 * 100)  # Convert to percentage
        
        print(f"  {all_vcfs[pipeline][i]}: Precision={precision*100:.1f}%, Recall={recall*100:.1f}%, F1={f1*100:.1f}%")

print("\nAll precision, recall, and F1 scores calculated successfully!")

Calculating precision, recall, and F1 scores...
Processing pipeline: mutect_yb_bwa
  g10_mutect_yb_bwa.vcf: Precision=61.8%, Recall=26.0%, F1=36.6%
  g1_mutect_yb_bwa.vcf: Precision=86.2%, Recall=78.1%, F1=82.0%
  g4_mutect_yb_bwa.vcf: Precision=28.1%, Recall=66.2%, F1=39.5%
  g11_mutect_yb_bwa.vcf: Precision=70.1%, Recall=8.7%, F1=15.5%
  g5_mutect_yb_bwa.vcf: Precision=84.8%, Recall=61.2%, F1=71.1%
  g3_mutect_yb_bwa.vcf: Precision=63.6%, Recall=46.6%, F1=53.8%
  g6_mutect_yb_bwa.vcf: Precision=82.5%, Recall=47.1%, F1=60.0%
  g2_mutect_yb_bwa.vcf: Precision=86.5%, Recall=77.8%, F1=81.9%
  g7_mutect_yb_bwa.vcf: Precision=86.3%, Recall=78.0%, F1=82.0%
  g9_mutect_yb_bwa.vcf: Precision=81.1%, Recall=22.5%, F1=35.2%
  g8_mutect_yb_bwa.vcf: Precision=82.5%, Recall=47.1%, F1=60.0%
Processing pipeline: mutect_yb_bowtie
  g6_mutect_yb_bowtie.vcf: Precision=87.2%, Recall=35.8%, F1=50.8%
  g3_mutect_yb_bowtie.vcf: Precision=70.2%, Recall=44.4%, F1=54.4%
  g9_mutect_yb_bowtie.vcf: Precision=87.

In [14]:
# Convert to DataFrames and create pivot tables for the new metrics
precision_df = pd.DataFrame(precision_data)
recall_df = pd.DataFrame(recall_data)
f1_df = pd.DataFrame(f1_data)

# Create pivot table for precision
pivot_precision = precision_df.pivot_table(
    index='Pipeline', 
    columns='Group', 
    values='Precision', 
    aggfunc='first'
)

# Create pivot table for recall
pivot_recall = recall_df.pivot_table(
    index='Pipeline', 
    columns='Group', 
    values='Recall', 
    aggfunc='first'
)

# Create pivot table for F1 scores
pivot_f1 = f1_df.pivot_table(
    index='Pipeline', 
    columns='Group', 
    values='F1_Score', 
    aggfunc='first'
)

print("Pivot tables for precision, recall, and F1 scores created successfully!")

Pivot tables for precision, recall, and F1 scores created successfully!


In [15]:
# Display the precision table
print("=== PRECISION TABLE (Percentages) ===")
display(pivot_precision)

print("\n=== RECALL TABLE (Percentages) ===")
display(pivot_recall)

print("\n=== F1 SCORE TABLE (Percentages) ===")
display(pivot_f1)

=== PRECISION TABLE (Percentages) ===


Group,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Pipeline,,,,,,,,,,,
Mutect-NB-BWA,77.753780,57.931034,67.659574,80.699301,59.266409,22.739459,83.916991,79.731544,83.951763,79.731544,85.090909
Mutect-NB-Bowtie,83.295455,14.018692,70.078740,89.798206,67.814371,25.490746,90.129870,84.948454,90.231092,84.948454,89.699571
Mutect-YB-BWA,86.216730,61.758691,70.138889,86.494253,63.572268,28.117002,84.844869,82.503771,86.285714,82.503771,81.055901
Mutect-YB-Bowtie,87.755102,22.021858,71.544715,91.326531,70.163488,32.228491,91.799787,87.211740,87.250996,87.211740,87.654321
SS-NB-BWA,30.114226,8.915094,8.915094,30.114226,9.258274,30.114226,30.114226,30.114226,30.114226,30.114226,30.114226
SS-NB-Bowtie,33.416459,11.324752,11.723535,33.416459,11.723535,33.416459,33.416459,33.416459,33.416459,33.416459,33.416459
SS-YB-BWA,36.921747,9.877202,9.877202,36.921747,10.301568,37.724868,36.921747,36.921747,36.921747,36.937716,36.937716
SS-YB-Bowtie,44.577626,13.640049,14.145988,44.577626,14.145988,44.577626,44.577626,44.577626,44.577626,44.577626,44.577626
Strelka-NB-BWA,42.785844,28.954802,28.295376,42.785844,29.757021,0.947053,42.785844,42.785844,42.805266,42.844162,42.844162



=== RECALL TABLE (Percentages) ===


Group,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Pipeline,,,,,,,,,,,
Mutect-NB-BWA,62.015504,21.705426,13.695090,49.698536,26.442722,80.361757,55.727821,51.162791,77.950043,51.162791,20.155039
Mutect-NB-Bowtie,63.135228,21.963824,22.997416,68.992248,39.018088,78.294574,29.888028,35.486649,73.987941,35.486649,18.001723
Mutect-YB-BWA,78.122308,26.012059,8.699397,77.777778,46.597761,66.236003,61.240310,47.114556,78.036176,47.114556,22.480620
Mutect-YB-Bowtie,48.148148,34.711456,15.159345,46.253230,44.358312,78.725237,74.246339,35.831180,75.452196,35.831180,24.461671
SS-NB-BWA,74.935401,65.116279,65.116279,74.935401,74.935401,74.935401,74.935401,74.935401,74.935401,74.935401,74.935401
SS-NB-Bowtie,69.250646,59.862188,69.250646,69.250646,69.250646,69.250646,69.250646,69.250646,69.250646,69.250646,69.250646
SS-YB-BWA,73.557278,63.738157,63.738157,73.557278,73.557278,61.412575,73.557278,73.557278,73.557278,73.557278,73.557278
SS-YB-Bowtie,67.269595,57.967270,67.269595,67.269595,67.269595,67.269595,67.269595,67.269595,67.269595,67.269595,67.269595
Strelka-NB-BWA,81.223084,70.628768,70.628768,81.223084,81.223084,88.802756,81.223084,81.223084,81.223084,81.223084,81.223084



=== F1 SCORE TABLE (Percentages) ===


Group,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Pipeline,,,,,,,,,,,
Mutect-NB-BWA,68.998563,31.578947,22.779370,61.513859,36.569387,35.448328,66.977226,62.329486,80.839661,62.329486,32.590529
Mutect-NB-Bowtie,71.827536,17.114094,34.630350,78.032148,49.535265,38.459911,44.890039,50.060753,81.306200,50.060753,29.985653
Mutect-YB-BWA,81.970176,36.606061,15.478927,81.904762,53.777336,39.476386,71.135568,59.978070,81.953867,59.978070,35.198921
Mutect-YB-Bowtie,62.180200,26.947509,25.017768,61.406518,54.353562,45.734301,82.095238,50.793651,80.923788,50.793651,38.249158
SS-NB-BWA,42.962963,15.683020,15.683020,42.962963,16.480394,42.962963,42.962963,42.962963,42.962963,42.962963,42.962963
SS-NB-Bowtie,45.079899,19.046314,20.052376,45.079899,20.052376,45.079899,45.079899,45.079899,45.079899,45.079899,45.079899
SS-YB-BWA,49.165227,17.103895,17.103895,49.165227,18.072162,46.738774,49.165227,49.165227,49.165227,49.179384,49.179384
SS-YB-Bowtie,53.621696,22.083675,23.376235,53.621696,23.376235,53.621696,53.621696,53.621696,53.621696,53.621696,53.621696
Strelka-NB-BWA,56.047548,41.071876,40.404040,56.047548,43.556582,1.874120,56.047548,56.047548,56.064209,56.097561,56.097561


In [16]:
# Create combined tables for each metric
print("Creating combined tables with variant counts and metrics...")

# Combined table for precision
combined_precision = {}
for pipeline in pipeline_names:
    combined_precision[pipeline] = {}
    for group in group_names:
        variant_count = pivot_variant_counts.loc[pipeline, group]
        precision_pct = pivot_precision.loc[pipeline, group]
        
        if pd.isna(variant_count) or pd.isna(precision_pct):
            combined_precision[pipeline][group] = "N/A"
        else:
            combined_precision[pipeline][group] = f"{variant_count} ({precision_pct:.1f}%)"

# Combined table for recall
combined_recall = {}
for pipeline in pipeline_names:
    combined_recall[pipeline] = {}
    for group in group_names:
        variant_count = pivot_variant_counts.loc[pipeline, group]
        recall_pct = pivot_recall.loc[pipeline, group]
        
        if pd.isna(variant_count) or pd.isna(recall_pct):
            combined_recall[pipeline][group] = "N/A"
        else:
            combined_recall[pipeline][group] = f"{variant_count} ({recall_pct:.1f}%)"

# Combined table for F1 scores
combined_f1 = {}
for pipeline in pipeline_names:
    combined_f1[pipeline] = {}
    for group in group_names:
        variant_count = pivot_variant_counts.loc[pipeline, group]
        f1_pct = pivot_f1.loc[pipeline, group]
        
        if pd.isna(variant_count) or pd.isna(f1_pct):
            combined_f1[pipeline][group] = "N/A"
        else:
            combined_f1[pipeline][group] = f"{variant_count} ({f1_pct:.1f}%)"

# Convert to DataFrames
combined_precision_df = pd.DataFrame(combined_precision).T
combined_recall_df = pd.DataFrame(combined_recall).T
combined_f1_df = pd.DataFrame(combined_f1).T

print("Combined tables created successfully!")

Creating combined tables with variant counts and metrics...
Combined tables created successfully!


In [17]:
# Display the combined tables
print("=== COMBINED TABLE: Variant Counts + Precision ===")
display(combined_precision_df)

print("\n=== COMBINED TABLE: Variant Counts + Recall ===")
display(combined_recall_df)

print("\n=== COMBINED TABLE: Variant Counts + F1 Scores ===")
display(combined_f1_df)

=== COMBINED TABLE: Variant Counts + Precision ===


,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Mutect-NB-BWA,926 (77.8%),435 (57.9%),235 (67.7%),715 (80.7%),518 (59.3%),4103 (22.7%),771 (83.9%),745 (79.7%),1078 (84.0%),745 (79.7%),275 (85.1%)
Mutect-NB-Bowtie,880 (83.3%),1819 (14.0%),381 (70.1%),892 (89.8%),668 (67.8%),3566 (25.5%),385 (90.1%),485 (84.9%),952 (90.2%),485 (84.9%),233 (89.7%)
Mutect-YB-BWA,1052 (86.2%),489 (61.8%),144 (70.1%),1044 (86.5%),851 (63.6%),2735 (28.1%),838 (84.8%),663 (82.5%),1050 (86.3%),663 (82.5%),322 (81.1%)
Mutect-YB-Bowtie,637 (87.8%),1830 (22.0%),246 (71.5%),588 (91.3%),734 (70.2%),2836 (32.2%),939 (91.8%),477 (87.2%),1004 (87.3%),477 (87.2%),324 (87.7%)
SS-NB-BWA,2889 (30.1%),8480 (8.9%),8480 (8.9%),2889 (30.1%),9397 (9.3%),2889 (30.1%),2889 (30.1%),2889 (30.1%),2889 (30.1%),2889 (30.1%),2889 (30.1%)
SS-NB-Bowtie,2406 (33.4%),6137 (11.3%),6858 (11.7%),2406 (33.4%),6858 (11.7%),2406 (33.4%),2406 (33.4%),2406 (33.4%),2406 (33.4%),2406 (33.4%),2406 (33.4%)
SS-YB-BWA,2313 (36.9%),7492 (9.9%),7492 (9.9%),2313 (36.9%),8290 (10.3%),1890 (37.7%),2313 (36.9%),2313 (36.9%),2313 (36.9%),2312 (36.9%),2312 (36.9%)
SS-YB-Bowtie,1752 (44.6%),4934 (13.6%),5521 (14.1%),1752 (44.6%),5521 (14.1%),1752 (44.6%),1752 (44.6%),1752 (44.6%),1752 (44.6%),1752 (44.6%),1752 (44.6%)
Strelka-NB-BWA,2204 (42.8%),2832 (29.0%),2898 (28.3%),2204 (42.8%),3169 (29.8%),108864 (0.9%),2204 (42.8%),2204 (42.8%),2203 (42.8%),2201 (42.8%),2201 (42.8%)
Strelka-NB-Bowtie,1313 (63.3%),107672 (0.8%),1839 (45.2%),1313 (63.3%),1786 (46.5%),90416 (1.1%),1313 (63.3%),1313 (63.3%),1313 (63.3%),1313 (63.3%),1313 (63.3%)



=== COMBINED TABLE: Variant Counts + Recall ===


,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Mutect-NB-BWA,926 (62.0%),435 (21.7%),235 (13.7%),715 (49.7%),518 (26.4%),4103 (80.4%),771 (55.7%),745 (51.2%),1078 (78.0%),745 (51.2%),275 (20.2%)
Mutect-NB-Bowtie,880 (63.1%),1819 (22.0%),381 (23.0%),892 (69.0%),668 (39.0%),3566 (78.3%),385 (29.9%),485 (35.5%),952 (74.0%),485 (35.5%),233 (18.0%)
Mutect-YB-BWA,1052 (78.1%),489 (26.0%),144 (8.7%),1044 (77.8%),851 (46.6%),2735 (66.2%),838 (61.2%),663 (47.1%),1050 (78.0%),663 (47.1%),322 (22.5%)
Mutect-YB-Bowtie,637 (48.1%),1830 (34.7%),246 (15.2%),588 (46.3%),734 (44.4%),2836 (78.7%),939 (74.2%),477 (35.8%),1004 (75.5%),477 (35.8%),324 (24.5%)
SS-NB-BWA,2889 (74.9%),8480 (65.1%),8480 (65.1%),2889 (74.9%),9397 (74.9%),2889 (74.9%),2889 (74.9%),2889 (74.9%),2889 (74.9%),2889 (74.9%),2889 (74.9%)
SS-NB-Bowtie,2406 (69.3%),6137 (59.9%),6858 (69.3%),2406 (69.3%),6858 (69.3%),2406 (69.3%),2406 (69.3%),2406 (69.3%),2406 (69.3%),2406 (69.3%),2406 (69.3%)
SS-YB-BWA,2313 (73.6%),7492 (63.7%),7492 (63.7%),2313 (73.6%),8290 (73.6%),1890 (61.4%),2313 (73.6%),2313 (73.6%),2313 (73.6%),2312 (73.6%),2312 (73.6%)
SS-YB-Bowtie,1752 (67.3%),4934 (58.0%),5521 (67.3%),1752 (67.3%),5521 (67.3%),1752 (67.3%),1752 (67.3%),1752 (67.3%),1752 (67.3%),1752 (67.3%),1752 (67.3%)
Strelka-NB-BWA,2204 (81.2%),2832 (70.6%),2898 (70.6%),2204 (81.2%),3169 (81.2%),108864 (88.8%),2204 (81.2%),2204 (81.2%),2203 (81.2%),2201 (81.2%),2201 (81.2%)
Strelka-NB-Bowtie,1313 (71.6%),107672 (75.4%),1839 (71.6%),1313 (71.6%),1786 (71.6%),90416 (86.5%),1313 (71.6%),1313 (71.6%),1313 (71.6%),1313 (71.6%),1313 (71.6%)



=== COMBINED TABLE: Variant Counts + F1 Scores ===


,G1,G10,G11,G2,G3,G4,G5,G6,G7,G8,G9
Mutect-NB-BWA,926 (69.0%),435 (31.6%),235 (22.8%),715 (61.5%),518 (36.6%),4103 (35.4%),771 (67.0%),745 (62.3%),1078 (80.8%),745 (62.3%),275 (32.6%)
Mutect-NB-Bowtie,880 (71.8%),1819 (17.1%),381 (34.6%),892 (78.0%),668 (49.5%),3566 (38.5%),385 (44.9%),485 (50.1%),952 (81.3%),485 (50.1%),233 (30.0%)
Mutect-YB-BWA,1052 (82.0%),489 (36.6%),144 (15.5%),1044 (81.9%),851 (53.8%),2735 (39.5%),838 (71.1%),663 (60.0%),1050 (82.0%),663 (60.0%),322 (35.2%)
Mutect-YB-Bowtie,637 (62.2%),1830 (26.9%),246 (25.0%),588 (61.4%),734 (54.4%),2836 (45.7%),939 (82.1%),477 (50.8%),1004 (80.9%),477 (50.8%),324 (38.2%)
SS-NB-BWA,2889 (43.0%),8480 (15.7%),8480 (15.7%),2889 (43.0%),9397 (16.5%),2889 (43.0%),2889 (43.0%),2889 (43.0%),2889 (43.0%),2889 (43.0%),2889 (43.0%)
SS-NB-Bowtie,2406 (45.1%),6137 (19.0%),6858 (20.1%),2406 (45.1%),6858 (20.1%),2406 (45.1%),2406 (45.1%),2406 (45.1%),2406 (45.1%),2406 (45.1%),2406 (45.1%)
SS-YB-BWA,2313 (49.2%),7492 (17.1%),7492 (17.1%),2313 (49.2%),8290 (18.1%),1890 (46.7%),2313 (49.2%),2313 (49.2%),2313 (49.2%),2312 (49.2%),2312 (49.2%)
SS-YB-Bowtie,1752 (53.6%),4934 (22.1%),5521 (23.4%),1752 (53.6%),5521 (23.4%),1752 (53.6%),1752 (53.6%),1752 (53.6%),1752 (53.6%),1752 (53.6%),1752 (53.6%)
Strelka-NB-BWA,2204 (56.0%),2832 (41.1%),2898 (40.4%),2204 (56.0%),3169 (43.6%),108864 (1.9%),2204 (56.0%),2204 (56.0%),2203 (56.1%),2201 (56.1%),2201 (56.1%)
Strelka-NB-Bowtie,1313 (67.2%),107672 (1.6%),1839 (55.4%),1313 (67.2%),1786 (56.4%),90416 (2.2%),1313 (67.2%),1313 (67.2%),1313 (67.2%),1313 (67.2%),1313 (67.2%)


In [18]:
# Save all tables to Excel file with multiple sheets (updated version)
print("Saving results to Excel file with all metrics...")

with pd.ExcelWriter("variant_counts_and_all_metrics.xlsx", engine='openpyxl') as writer:
    # Original sheets
    combined_df.to_excel(writer, sheet_name='Combined_Results_Jaccard')
    
    # New metric sheets
    combined_precision_df.to_excel(writer, sheet_name='Combined_Results_Precision')
    combined_recall_df.to_excel(writer, sheet_name='Combined_Results_Recall')
    combined_f1_df.to_excel(writer, sheet_name='Combined_Results_F1')
    
    # Individual metric sheets
    pivot_precision.to_excel(writer, sheet_name='Precision_Only')
    pivot_recall.to_excel(writer, sheet_name='Recall_Only')
    pivot_f1.to_excel(writer, sheet_name='F1_Scores_Only')
    
    # Original individual sheets
    pivot_variant_counts.to_excel(writer, sheet_name='Variant_Counts_Only')
    pivot_jaccard.to_excel(writer, sheet_name='Jaccard_Similarities_Only')
    
    # Raw data sheets
    variant_counts_df.to_excel(writer, sheet_name='Raw_Data_Jaccard', index=False)
    precision_df.to_excel(writer, sheet_name='Raw_Data_Precision', index=False)
    recall_df.to_excel(writer, sheet_name='Raw_Data_Recall', index=False)
    f1_df.to_excel(writer, sheet_name='Raw_Data_F1', index=False)

print("Results saved to 'variant_counts_and_all_metrics.xlsx' with comprehensive metric sheets!")

Saving results to Excel file with all metrics...
Results saved to 'variant_counts_and_all_metrics.xlsx' with comprehensive metric sheets!


In [19]:
# Summary statistics for all metrics
print("=== COMPREHENSIVE SUMMARY STATISTICS ===\n")

# Jaccard Similarity Statistics
jaccard_values = variant_counts_df['Jaccard Similarity'].dropna()
print("Jaccard Similarity Statistics:")
print(f"  Mean: {jaccard_values.mean():.2f}%")
print(f"  Median: {jaccard_values.median():.2f}%")
print(f"  Min: {jaccard_values.min():.2f}%")
print(f"  Max: {jaccard_values.max():.2f}%")
print(f"  Std Dev: {jaccard_values.std():.2f}%")

# Precision Statistics
precision_values = precision_df['Precision'].dropna()
print(f"\nPrecision Statistics:")
print(f"  Mean: {precision_values.mean():.2f}%")
print(f"  Median: {precision_values.median():.2f}%")
print(f"  Min: {precision_values.min():.2f}%")
print(f"  Max: {precision_values.max():.2f}%")
print(f"  Std Dev: {precision_values.std():.2f}%")

# Recall Statistics
recall_values = recall_df['Recall'].dropna()
print(f"\nRecall Statistics:")
print(f"  Mean: {recall_values.mean():.2f}%")
print(f"  Median: {recall_values.median():.2f}%")
print(f"  Min: {recall_values.min():.2f}%")
print(f"  Max: {recall_values.max():.2f}%")
print(f"  Std Dev: {recall_values.std():.2f}%")

# F1 Score Statistics
f1_values = f1_df['F1_Score'].dropna()
print(f"\nF1 Score Statistics:")
print(f"  Mean: {f1_values.mean():.2f}%")
print(f"  Median: {f1_values.median():.2f}%")
print(f"  Min: {f1_values.min():.2f}%")
print(f"  Max: {f1_values.max():.2f}%")
print(f"  Std Dev: {f1_values.std():.2f}%")

# Find best and worst performing pipelines for each metric
print(f"\n=== TOP PERFORMERS ===")
print(f"Best Jaccard: {variant_counts_df.loc[jaccard_values.idxmax(), 'Pipeline']} - {variant_counts_df.loc[jaccard_values.idxmax(), 'Group']} ({jaccard_values.max():.1f}%)")
print(f"Best Precision: {precision_df.loc[precision_values.idxmax(), 'Pipeline']} - {precision_df.loc[precision_values.idxmax(), 'Group']} ({precision_values.max():.1f}%)")
print(f"Best Recall: {recall_df.loc[recall_values.idxmax(), 'Pipeline']} - {recall_df.loc[recall_values.idxmax(), 'Group']} ({recall_values.max():.1f}%)")
print(f"Best F1: {f1_df.loc[f1_values.idxmax(), 'Pipeline']} - {f1_df.loc[f1_values.idxmax(), 'Group']} ({f1_values.max():.1f}%)")

=== COMPREHENSIVE SUMMARY STATISTICS ===

Jaccard Similarity Statistics:
  Mean: 33.71%
  Median: 32.76%
  Min: 0.81%
  Max: 69.63%
  Std Dev: 16.57%

Precision Statistics:
  Mean: 48.92%
  Median: 44.58%
  Min: 0.81%
  Max: 91.80%
  Std Dev: 26.03%

Recall Statistics:
  Mean: 64.36%
  Median: 71.15%
  Min: 8.70%
  Max: 88.80%
  Std Dev: 18.31%

F1 Score Statistics:
  Mean: 48.07%
  Median: 49.36%
  Min: 1.61%
  Max: 82.10%
  Std Dev: 19.38%

=== TOP PERFORMERS ===
Best Jaccard: Mutect-YB-Bowtie - G5 (69.6%)
Best Precision: Mutect-YB-Bowtie - G5 (91.8%)
Best Recall: Strelka-NB-BWA - G4 (88.8%)
Best F1: Mutect-YB-Bowtie - G5 (82.1%)
